# Face Recognition Based Attendance System

Face Recognition has become an emerging technology in today’s world. It has been widely spread throughout the last few years in a drastic way. There have been some big improvements in the last few years which has made it so much popular that now it is being widely used for commercial as well as security purpose. Tracking the presence of a person is a very big problem, and an Attendance system based on Face Recognition can act as a real-world solution and can provide simplicity in handling attendance records.<br>
The system is developed for deploying an easy and secure way of taking down attendance. The GUI gives you options to register users and take their attendance.<br>
First, the system requires registration of new users that captures images of the particular individual using a live camera and stores the images as a dataset to train the recognizer. The Registration process takes an ID and Name as input for individual new user.<br>
The Attendance taking system needs the already registered users to stand in front of the camera to recognize the face and notes down the attendance with time and date.


## Advantages:
* The system can be used for security purposes in organizations and in secured zones.
* The system stores the faces that are detected and automatically marks attendance.
* The system is convenient and secure for the users.
* It saves their time and effort.
* It makes it easy to maintain attendance records.


## Disadvantages:
* The system doesn’t recognize properly in poor light so may give false results.
* It can only detect a face from a limited distance.
* Facial recognition systems cannot account for faces that are captured at angles other than straight into the capturing camera. It makes the attendance marking process slower and less efficient.


In [1]:
# Required Packages Imported
import numpy as np
import cv2
import csv
from PIL import Image
import os
import datetime
import time
import pandas as pd
from tkinter import messagebox
from tkinter import *
from PIL import ImageTk, Image
from functools import partial

## Face Detection:
The Capture Function is defined to take images of new user.<br>
It reads the face, captures images and stores them as dataset. It also updates the ID and Name of new user.

In [2]:
def face_detector(img):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.5, minNeighbors=5)
    if faces is():
        return None
    else:
        return faces

def capture(id, name):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
    cap = cv2.VideoCapture(0)
    count=0
    while(True):
        ret, frame = cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces=face_detector(gray)
        if faces is not None:
            for (x,y,w,h) in faces:
                #print(x, y, w, h)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                face_crop = gray[y:y+h, x:x+w]
                cv2.imwrite("face_data/"+str(name)+'_'+str(id)+"_"+str(count)+".jpg",face_crop)
                count+=1
                cv2.imshow('frame', frame)
                if count == 50:
                    break

            if count == 50:
                break
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    cap.release()
    row = [id , name]
    with open('Attendance_Record\AttendanceDetails.csv','a+') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(row)
    csvFile.close()
    cv2.destroyAllWindows()

## Training:
The Training function is defined to train our Recognizer Model.<br>
It reads all the images and its label from Database and trains the Recognizer Model. It saves the model for further use.

In [3]:
def getImagesData(path):
    #path to all the images in the folder
    imagesPath = [os.path.join(path,f) for f in os.listdir(path)]

    #List to contain faces
    faces = []
    #List to contain Ids
    ids = []

    for imagePath in imagesPath:
        pilImage = Image.open(imagePath).convert('L')
        imgnp = np.array(pilImage, 'uint8')

        Id = int(os.path.split(imagePath)[-1].split("_")[1])

        faces.append(imgnp)
        ids.append(Id)

    return faces, ids

def Training():
    recognizer = cv2.face_LBPHFaceRecognizer.create()
    detector = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
    faces, ids = getImagesData('face_data')
    recognizer.train(faces, np.array(ids))
    recognizer.save('Trained_Recognizer/trainner.yml')


## Face Recognition:
The CheckImage function is defined to take attendance.<br>
It recognizes the face by the use of our trained Recognizer Model, and stores the ID, Name, Time and Date of Attendance for each recognized face. It shows unknown if the face is not recognized.

In [4]:
def face_detector(img):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.5, minNeighbors=5)
    if faces is ():
        return None
    else:
        return faces

def CheckImage():
    recognizer = cv2.face.LBPHFaceRecognizer_create()#cv2.createLBPHFaceRecognizer()
    recognizer.read("Trained_Recognizer/trainner.yml")
    faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
    df=pd.read_csv("Attendance_Record/AttendanceDetails.csv")
    cam = cv2.VideoCapture(0)
    font = cv2.FONT_HERSHEY_SIMPLEX
    col_names =  ['Id','Name','Date','Time']
    attendance = pd.DataFrame(columns = col_names)
    while True:
        ret, frame =cam.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces=face_detector(gray)
        if faces is not None:
                for (x,y,w,h) in faces:
                    print(x, y, w, h)
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                    Id, conf = recognizer.predict(gray[y:y+h,x:x+w])
                    if(conf < 50):
                        ts = time.time()
                        date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
                        timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
                        aa=df.loc[df['Id'] == Id]['Name'].values
                        tt=str(Id)+"-"+aa
                        attendance.loc[len(attendance)] = [Id,aa,date,timeStamp]
                    else:
                        Id='Unknown'
                        tt=str(Id)
                    cv2.putText(frame,str(tt),(x,y+h), font, 1,(255,255,255),2)
        attendance=attendance.drop_duplicates(subset=['Id'],keep='first')
        cv2.imshow('frame', frame)
        if (cv2.waitKey(1)==ord('q')):
            break
    ts = time.time()
    date = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d')
    timeStamp = datetime.datetime.fromtimestamp(ts).strftime('%H:%M:%S')
    Hour,Minute,Second=timeStamp.split(":")
    fileName="Attendance_Record/Attendance_"+date+"_"+Hour+"-"+Minute+"-"+Second+".csv"
    attendance.to_csv(fileName,index=False)
    cam.release()
    cv2.destroyAllWindows()
    #print(attendance)
    return attendance

## GUI:
It handles all the functions and when to run them.<br>
It shows Notification after the new user is registered and also displays the attendance when face is recognized.

In [5]:
class Start:
    def __init__(self, master):
        self.master = master
        self.frame = Frame(self.master)
        self.master.geometry("700x600")
        self.master.title("AI Solution Attendance System")

        self.logo = ImageTk.PhotoImage(Image.open("Resources/logo_image.png"))
        self.logo_img = Label(self.frame, image = self.logo)
        self.complabel=Label(self.frame, text="AIS Solutions PVT LTD", font=("Comic sans MS",24,"bold"))
        self.heading=Label(self.frame, text="Attendance Management System", font=("Comic sans MS",24,"bold"))
        self.take_attendence=Button(self.frame,
                               text="Take Attendance",
                               width="30",
                               height="2",
                               background="#000000",
                               foreground="#ffffff",
                               activeforeground="#ff0000",
                               command=self.take_attendence
                               )
        self.register_user=Button(self.frame,
                             text="Register New User",
                             width="30",
                             height="2",
                             background="#000000",
                             foreground="#ffffff",
                             activeforeground="#ff0000",
                             command=self.new_window
                             )
        self.exit_btn=Button(self.frame,
                        text="Exit",
                        width="30",
                        height="2",
                        background="#000000",
                        foreground="#ffffff",
                        activeforeground="#ff0000",
                        command=self.exit_app
                        )
        self.notif1 = Label(self.frame,text="",font=("Arial sans MS",14,"bold"),fg="red")
        self.logo_img.pack()
        self.complabel.pack(pady=(0,0))
        self.heading.pack(pady=(0,30))
        self.take_attendence.pack(pady=(0,30))
        self.register_user.pack(pady=(0,30))
        self.exit_btn.pack(pady=(0,30))
        self.notif1.pack(pady=(30,0))
        self.frame.pack()

    def new_window(self):
        self.newWindow = Toplevel(self.master)
        self.app = register_user(self.newWindow)

    def take_attendence(self):
        attendance = CheckImage()
        msg = attendance
        self.notif1.configure(text=msg)

    def exit_app(self):
        self.master.destroy()

class register_user:
    def __init__(self, master):
        self.master = master
        self.frame = Frame(self.master)
        self.master.title("Register New User")
        self.master.geometry("500x400")
        self.id_label=Label(self.frame,text="User ID", font=("Arial sans MS",14,"bold"))
        self.id_input=Entry(self.frame,bd=2,width="30",font=("Arial sans MS",14))

        self.name_label=Label(self.frame,text="User Name",font=("Arial sans MS",14,"bold"))
        self.name_input=Entry(self.frame,bd=2,width="30",font=("Arial sans MS",14))
        self.take_face = Button(self.frame,
                                 text="Take Face",
                                 width="30",
                                 height="2",
                                 background="#000000",
                                 foreground="#ffffff",
                                 activeforeground="#ff0000",
                                 command=lambda: self.face_capture(self.id_input.get(), self.name_input.get())
                                 )
        self.complete_registration = Button(self.frame,
                           text="Register",
                           width="30",
                           height="2",
                           background="#000000",
                           foreground="#ffffff",
                           activeforeground="#ff0000",
                           command=self.train_images
                           )
        self.notif1 = Label(self.frame,text="",font=("Arial sans MS",14,"bold"),fg="red")

        self.id_label.pack(pady=(30,0))
        self.id_input.pack()
        self.name_label.pack(pady=(30,0))
        self.name_input.pack()
        self.take_face.pack(pady="30")
        self.complete_registration.pack()
        self.notif1.pack(pady=(30,0))
        self.frame.pack()

    def close_windows(self):
        self.master.destroy()

    def is_number(self,s):
        try:
            float(s)
            return True
        except ValueError:
            pass

        try:
            import unicodedata
            unicodedata.numeric(s)
            return True
        except (TypeError, ValueError):
            pass

        return False

    def face_capture(self, id, name):
        self.id = id
        self.name = name
        if (self.is_number(self.id) and self.name.isalpha()):
            capture(self.id, self.name)
            msg = "Images Recorded for ID: "+self.id+", Name: "+self.name
            self.notif1.configure(text=msg)
        else:
            if self.is_number(self.id):
                msg = "Notification: Invalid Name"
                self.notif1.configure(text=msg)
            if self.name.isalpha():
                msg = "Notification: ID should be number"
                self.notif1.configure(text=msg)

    def train_images(self):
        Training()
        self.master.destroy()



def main():
    root = Tk()
    app = Start(root)
    root.mainloop()

if __name__ == '__main__':
    main()


292 175 152 152
289 176 152 152
289 177 152 152
288 177 152 152
284 177 152 152
284 178 152 152
278 178 152 152
274 180 152 152
274 185 152 152
268 181 152 152
267 181 152 152
266 183 152 152
266 183 152 152
265 181 152 152
262 185 152 152
262 187 152 152
220 152 228 228
258 188 152 152
218 151 228 228
218 151 228 228
216 151 228 228
216 150 228 228
216 148 228 228
217 152 228 228
217 153 228 228
218 147 228 228
218 147 228 228
216 148 228 228
216 140 228 228
215 140 228 228
215 138 228 228
218 144 228 228
220 146 228 228
218 147 228 228
220 142 228 228
219 142 228 228
216 142 228 228
216 142 228 228
214 142 228 228
214 142 228 228
213 147 228 228
211 148 228 228
213 145 228 228
208 143 228 228
206 141 228 228
205 142 228 228
205 146 228 228
205 148 228 228
205 146 228 228
202 148 228 228
204 149 228 228
201 149 228 228
204 152 228 228
204 149 228 228
204 149 228 228
204 149 228 228
202 150 228 228
200 148 228 228
200 148 228 228
201 147 228 228
202 148 228 228
199 148 228 228
204 149 